In [41]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import metrics
import matplotlib.pyplot as plt 
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.linear_model import Lasso,Ridge
from sklearn.preprocessing import MinMaxScaler 
from sklearn.linear_model import BayesianRidge
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import MultiTaskLasso
from sklearn.linear_model import PassiveAggressiveRegressor
from sklearn.linear_model import ElasticNet,MultiTaskElasticNet
from sklearn.model_selection import train_test_split as tts
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.experimental import enable_hist_gradient_boosting 
from sklearn.model_selection import RepeatedStratifiedKFold,cross_val_score
from sklearn.ensemble import RandomForestRegressor,BaggingRegressor,AdaBoostRegressor


In [42]:
df=pd.read_csv("/content/drive/MyDrive/machine hack/fortnight_hackathon/train.csv")
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,52,Private,98926,HS-grad,9,Never-married,Other-service,Unmarried,White,Male,-3,0,44,United-States,<=50K
1,76,Self-emp-not-inc,132607,Bachelors,12,Married-civ-spouse,Sales,Husband,White,Male,7527,0,44,United-States,>50K
2,40,Private,243258,Bachelors,13,Married-civ-spouse,Transport-moving,Husband,Amer-Indian-Eskimo,Male,19,0,57,United-States,>50K
3,76,State-gov,181259,Bachelors,12,Married-civ-spouse,Prof-specialty,Husband,White,Male,7720,0,44,United-States,>50K
4,36,Self-emp-inc,115379,Masters,13,Divorced,Exec-managerial,Not-in-family,White,Male,12,0,58,United-States,<=50K


In [43]:
for col in df.select_dtypes(include='object').columns:
  df[col+'__mean_hours']=df[col].replace(dict(df.groupby([col])['hours-per-week'].mean()))
  df=df.drop(col,1)
    
df.head()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,workclass__mean_hours,education__mean_hours,marital-status__mean_hours,occupation__mean_hours,relationship__mean_hours,race__mean_hours,sex__mean_hours,native-country__mean_hours,income__mean_hours
0,52,98926,9,-3,0,44,38.362821,39.008788,37.618747,34.505155,36.593197,41.069942,43.03617,40.377938,38.429607
1,76,132607,12,7527,0,44,45.182907,43.956248,43.229936,41.683284,44.296343,41.069942,43.03617,40.377938,44.327805
2,40,243258,13,19,0,57,38.362821,43.956248,43.229936,41.029114,44.296343,40.470468,43.03617,40.377938,44.327805
3,76,181259,12,7720,0,44,39.225885,43.956248,43.229936,44.546594,44.296343,41.069942,43.03617,40.377938,44.327805
4,36,115379,13,12,0,58,46.083624,43.543039,38.838339,44.430331,41.455951,41.069942,43.03617,40.377938,38.429607


In [44]:
df=pd.get_dummies(df,drop_first=True)

df.head()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,workclass__mean_hours,education__mean_hours,marital-status__mean_hours,occupation__mean_hours,relationship__mean_hours,race__mean_hours,sex__mean_hours,native-country__mean_hours,income__mean_hours
0,52,98926,9,-3,0,44,38.362821,39.008788,37.618747,34.505155,36.593197,41.069942,43.03617,40.377938,38.429607
1,76,132607,12,7527,0,44,45.182907,43.956248,43.229936,41.683284,44.296343,41.069942,43.03617,40.377938,44.327805
2,40,243258,13,19,0,57,38.362821,43.956248,43.229936,41.029114,44.296343,40.470468,43.03617,40.377938,44.327805
3,76,181259,12,7720,0,44,39.225885,43.956248,43.229936,44.546594,44.296343,41.069942,43.03617,40.377938,44.327805
4,36,115379,13,12,0,58,46.083624,43.543039,38.838339,44.430331,41.455951,41.069942,43.03617,40.377938,38.429607


In [45]:
x=df.drop('hours-per-week',1)
y=df['hours-per-week']
print(x.shape)

(18944, 14)


# models

In [46]:
mms=MinMaxScaler()
x_=mms.fit_transform(x)

Poly_model=Pipeline([('poly', PolynomialFeatures(degree=2)),
                   ('linear', LinearRegression(fit_intercept=False))])

regressor={'LinearRegression':LinearRegression(),
          'Lasso':Lasso(),
           'Ridge':Ridge(),
           'ElasticNet':ElasticNet(),
           'BayesianRidge':BayesianRidge(),
           'PassiveAggressiveRegressor':PassiveAggressiveRegressor(),
          'PolyModel':Poly_model}

cv=RepeatedStratifiedKFold(n_splits=5,n_repeats=2,random_state=0)

for model_name,model in zip(regressor.keys(),regressor.items()):
    score=cross_val_score(model[1] ,x_,y,scoring='neg_root_mean_squared_error',cv=3)
    print(model_name,'\t',np.mean(score).round(3))

LinearRegression 	 -9.819
Lasso 	 -10.639
Ridge 	 -9.819
ElasticNet 	 -10.618
BayesianRidge 	 -9.819
PassiveAggressiveRegressor 	 -10.859
PolyModel 	 -9.728


In [47]:
tree_based_regressor={'GradientBoostingRegressor':GradientBoostingRegressor(),
            'HistGradientBoostingRegressor':HistGradientBoostingRegressor(),
            'XGBRegressor':XGBRegressor(),
            'LGBMRegressor':LGBMRegressor(),
            'BaggingRegressor':BaggingRegressor(),
            'AdaBoostRegressor':AdaBoostRegressor(),
            'RandomForestRegressor':RandomForestRegressor(),
            'CatBoostRegressor':CatBoostRegressor(verbose=0)}
cv=RepeatedStratifiedKFold(n_splits=5,n_repeats=2,random_state=0)

for model_name,model in zip(tree_based_regressor.keys(),tree_based_regressor.items()):
    score=cross_val_score(model[1] ,x,y,scoring='neg_root_mean_squared_error',cv=3)
    print(model_name,'\t',np.mean(score).round(3))


GradientBoostingRegressor 	 -9.683
HistGradientBoostingRegressor 	 -9.778
[09:48:18] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:48:19] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:48:20] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
XGBRegressor 	 -9.679
LGBMRegressor 	 -9.765
BaggingRegressor 	 -10.506
AdaBoostRegressor 	 -9.865
RandomForestRegressor 	 -10.039
CatBoostRegressor 	 -9.794


# Voting

In [48]:
from sklearn.ensemble import VotingRegressor,VotingClassifier

def voting(list_of_models):
    estimators=[]
    for mi,m in enumerate(list_of_models):
        estimators.append((str(mi),m))
    return VotingRegressor(estimators=estimators)

model=voting([Poly_model,
              HistGradientBoostingRegressor(),
              LGBMRegressor(),
              XGBRegressor(),
              GradientBoostingRegressor()])

score=cross_val_score(model,x,y,scoring='neg_root_mean_squared_error',cv=3)
print('voting','\t',np.mean(score).round(3))

[09:48:52] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:48:56] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:48:59] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
voting 	 -9.675


In [49]:
model.fit(x,y)

[09:49:03] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


VotingRegressor(estimators=[('0',
                             Pipeline(memory=None,
                                      steps=[('poly',
                                              PolynomialFeatures(degree=2,
                                                                 include_bias=True,
                                                                 interaction_only=False,
                                                                 order='C')),
                                             ('linear',
                                              LinearRegression(copy_X=True,
                                                               fit_intercept=False,
                                                               n_jobs=None,
                                                               normalize=False))],
                                      verbose=False)),
                            ('1',
                             HistGradientBoostingRegressor(l2_regulariz

# Test

In [50]:
test_df=pd.read_csv("/content/drive/MyDrive/machine hack/fortnight_hackathon/test.csv")
test_df.drop('hours-per-week',1,inplace=True)
test_df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,native-country,income
0,40,Without-pay,106899,HS-grad,8,Never-married,Adm-clerical,Own-child,White,Male,0,0,United-States,<=50K
1,38,Private,157191,HS-grad,8,Married-civ-spouse,Craft-repair,Husband,Black,Male,0,0,United-States,<=50K
2,76,Self-emp-not-inc,102547,Some-college,10,Widowed,Sales,Not-in-family,Asian-Pac-Islander,Female,32,0,United-States,<=50K
3,38,Self-emp-not-inc,44426,HS-grad,9,Never-married,Sales,Unmarried,White,Male,30,0,United-States,>50K
4,62,Without-pay,33230,HS-grad,9,Married-civ-spouse,Sales,Husband,White,Male,-4,0,United-States,<=50K


In [51]:
df=pd.read_csv("/content/drive/MyDrive/machine hack/fortnight_hackathon/train.csv")

In [52]:
for col in df.select_dtypes(include='object').columns:
  test_df[col+'__mean_hours']=test_df[col].replace(dict(df.groupby([col])['hours-per-week'].mean()))
  test_df=test_df.drop(col,1)
test_df.head()

,age,fnlwgt,education-num,capital-gain,capital-loss,workclass__mean_hours,education__mean_hours,marital-status__mean_hours,occupation__mean_hours,relationship__mean_hours,race__mean_hours,sex__mean_hours,native-country__mean_hours,income__mean_hours
0,40,106899,8,0,0,39.593023,39.008788,37.618747,37.691180,35.045159,41.069942,43.036170,40.377938,38.429607
1,38,157191,8,0,0,38.362821,39.008788,43.229936,41.743819,44.296343,37.088640,43.036170,40.377938,38.429607
2,76,102547,10,32,0,45.182907,37.244860,34.801282,41.683284,41.455951,39.932870,36.311428,40.377938,38.429607
3,38,44426,9,30,0,45.182907,39.008788,37.618747,41.683284,36.593197,41.069942,43.036170,40.377938,44.327805
4,62,33230,9,-4,0,39.593023,39.008788,43.229936,41.683284,44.296343,41.069942,43.036170,40.377938,38.429607


In [53]:
pred=model.predict(test_df)

In [54]:
pd.DataFrame({'hours-per-week':pred}).to_csv('mean_1.csv', index=False)